# Ridge Regression
This notebook contains the code for running multiple models on the ML Higgs boson data using Ridge regression with normal equations.

In [40]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from helpers import *
from implementations import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load train and test data

In [41]:
DATA_TRAIN_PATH = '../data/train.csv'
y_train, X_train, ids = load_csv_data(DATA_TRAIN_PATH)

In [42]:
DATA_TEST_PATH = '../data/test.csv'
_, X_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [43]:
X_train.shape

(250000, 30)

## Baseline model using raw data
First let's run Ridge regression algorithm on our raw data without doing any preprocessing. We will use K-fold cross validation to report the metrics on the test data and grid search to tune our hyperparameters.

In [44]:
param_grid = {
    'lambda_': np.logspace(-4,0,5) ,
    'max_iters': 1000,
    'gamma': [0.01, 0.1]
}
metrics, params = ridge_regression_cv(y_train, X_train, param_grid=param_grid, transform=False)

In [45]:
metrics, params

({'loss': 0.3397855566120538,
  'accuracy': 74.42840000000001,
  'f1_score': 0.5688096990110691},
 {'lambda_': 0.0001, 'max_iters': 1000, 'gamma': 0.01})

Ridge regression seems to be working decently with raw data.

## Baseline model using lightly feature engineered data
Now let's now preprocess our data a bit to handle the missing values (-999) in various ways.

### All features with NaN values imputed
First let's impute all missing values with median of their respective columns. So we will set the `imputable_th` to `1` which means impute all columns with a nan value ratio less than 1, or in other words all columns.

In [46]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=1, encodable_th=0)

In [47]:
tX_train.shape, tX_test.shape

((236483, 31), (568238, 31))

We have now all the columns imputed and plus one more column for the bias.

In [48]:
param_grid = {
    'lambda_': np.logspace(-4,0,5) ,
    'max_iters': 1000,
    'gamma': [0.01, 0.1]
}
metrics, params = ridge_regression_cv(ty_train, tX_train, param_grid=param_grid, transform=False)

In [49]:
metrics, params

({'loss': 0.3256406318235878,
  'accuracy': 76.44029093369419,
  'f1_score': 0.6129394851260843},
 {'lambda_': 0.0001, 'max_iters': 1000, 'gamma': 0.01})

Accuracy, f1_score and loss have gone up, compared to the raw values, indicating that this pre-processing is useful. 

### All features with NaN values encoded
Now let's instead encode these features with NaN values into new indicator features where the new feature takes on a value of 1 if the value for the feature is missing, otherwise 0.

In [50]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0, encodable_th=1)

In [51]:
tX_train.shape, tX_test.shape

((243430, 31), (568238, 31))

In [52]:
param_grid = {
    'lambda_': np.logspace(-4,0,5) ,
    'max_iters': 1000,
    'gamma': [0.01, 0.1]
}
metrics, params = ridge_regression_cv(ty_train, tX_train, param_grid=param_grid, transform=False)

In [53]:
metrics, params

({'loss': 0.33508274817893047,
  'accuracy': 75.14151912254036,
  'f1_score': 0.604671339546958},
 {'lambda_': 0.0001, 'max_iters': 1000, 'gamma': 0.01})

Accuracy, f1_score and loss have gone up, compared to the raw values, indicating that this pre-processing is useful but it is lower than only imputing values. This indicates that a mix of the two might be more useful 

### Mixed imputing and encoding approach
Finally,  let's try a more reasonable approach to the imputing and encoding scheme. As we saw in the exploration notebook, we have some features that have less than 15% of them missing, some around 40% and some more than 70%. Let's impute the columns in the first group, encode the ones in the second group and drop completely the ones in the third group.


In [54]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=0.3, encodable_th=0.7)

In [55]:
tX_train.shape, tX_test.shape

((242240, 24), (568238, 24))

In [56]:
param_grid = {
    'lambda_': np.logspace(-4,0,5) ,
    'max_iters': 1000,
    'gamma': [0.01, 0.1]
}
metrics, params = ridge_regression_cv(ty_train, tX_train, param_grid=param_grid, transform=False)

In [57]:
metrics, params

({'loss': 0.3360989798895204,
  'accuracy': 75.44129788639366,
  'f1_score': 0.61151010562385},
 {'lambda_': 0.0001, 'max_iters': 1000, 'gamma': 0.01})

Seems like we get the best performance when we impute all of the NaN values. Let's continue our feature engineering with these preprocessing thresholds fixed.

## Baseline model using heavily feature engineered data
In this step, we are going to apply more feature engineering. First, we will apply polynomial features of some degree that we will tune through grid search and cross validation.

In [58]:
tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=1, encodable_th=0)

In [59]:
tX_train.shape, tX_test.shape

((236483, 31), (568238, 31))

In [60]:
param_grid = {
    'lambda_': np.logspace(-4,0,5) ,
    'degree': list(range(1, 4)),
    'max_iters': 1000,
    'gamma': [0.01, 0.1],
    'cont_features': [cont_features]
}
metrics, params = ridge_regression_cv(ty_train, tX_train, param_grid=param_grid)

In [61]:
metrics, params

({'loss': 0.2940676846377305,
  'accuracy': 79.87271650879568,
  'f1_score': 0.6786819752057794},
 {'lambda_': 0.0001,
  'degree': 3,
  'max_iters': 1000,
  'gamma': 0.01,
  'cont_features': (1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30)})

It looks like polynomial feature expansion is useful for ridge regression, as the degree =3 was chosen and accuracy has gone up.

Next, we are going to split our datasets based on the number of jets (`PRI_jet_num`) and create 3 subsets of the data for observations with 0, 1 and more than 1 jet respectively. Each subset will also only have the relevant columns (based on the original paper) All other missing values in the new subsets will be imputed with median values.

In [62]:
X_train_zero, y_train_zero, X_train_one, y_train_one, X_train_many, y_train_many = split_by_jet_num(DATA_TRAIN_PATH, X_train, y_train)
X_test_zero, ids_test_zero, X_test_one, ids_test_one, X_test_many, ids_test_many = split_by_jet_num(DATA_TRAIN_PATH, X_test, ids_test)

In [63]:
X_train_zero.shape, X_train_one.shape, X_train_many.shape

((99913, 15), (77544, 22), (72543, 29))

In [64]:
def train(X_train, y_train, X_test):
    tX_train, ty_train, tX_test, ty_test, cont_features = preprocess(X_train, y_train, X_test, imputable_th=1, encodable_th=0)
    
    param_grid = {
    'lambda_': np.logspace(-4,0,5) ,
    'degree': list(range(1, 4)),
    'max_iters': 1000,
    'gamma': [0.01, 0.1],
    'cont_features': [cont_features]
}
    metrics, params = ridge_regression_cv(ty_train, tX_train, param_grid=param_grid)
    return metrics, params

In [65]:
metrics_zero, params_zero = train(X_train_zero, y_train_zero, X_test_zero)

In [66]:
metrics_one, params_one = train(X_train_one, y_train_one, X_test_one)

In [67]:
metrics_many, params_many = train(X_train_many, y_train_many, X_test_many)

In [68]:
metrics_zero, params_zero

({'loss': 0.2436432880326947,
  'accuracy': 83.19917758930868,
  'f1_score': 0.6209485184651606},
 {'lambda_': 0.0001,
  'degree': 3,
  'max_iters': 1000,
  'gamma': 0.01,
  'cont_features': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)})

In [69]:
metrics_one, params_one

({'loss': 0.31804964032282496,
  'accuracy': 78.14888535031847,
  'f1_score': 0.6755627506086908},
 {'lambda_': 0.0001,
  'degree': 3,
  'max_iters': 1000,
  'gamma': 0.01,
  'cont_features': (1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22)})

In [70]:
metrics_many, params_many

({'loss': 0.2968017286941423,
  'accuracy': 80.31111423945943,
  'f1_score': 0.7802669268659626},
 {'lambda_': 0.0001,
  'degree': 3,
  'max_iters': 1000,
  'gamma': 0.01,
  'cont_features': (1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29)})

In [72]:
a = X_train_zero.shape[0]
b =  X_train_one.shape[0] 
c = X_train_many.shape[0]
avg_accuracy = ((metrics_zero['accuracy']*a) +  (metrics_one['accuracy']*b) + (metrics_many['accuracy']*c))/(a+b+c)

print(f"Average accuracy with jet_num training is {avg_accuracy}")

Average accuracy with jet_num training is 80.7946630254352


Splitting by jet_num lets us arrive in the 80% accuracy territory.